# Process other cycling infrastructure sources

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [ ]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
export_fp = Path(config['project_directory']) / 'Cycling_Infra_Dating'

In [ ]:
studyarea_geo = gpd.read_file(config['studyarea']).to_crs('epsg:4326')

# City of Atlanta / Atlanta Regional Commission / Garber Processing
NOTE: There are some data quality issues with the CoA and ARC datasets in that sometimes facilities are marked as there when they aren't. Refine this in the future.

## City of Atlanta 
Has year installed date and other notes. Contains sharrows. Also contains "planned" infrastructure that may not have actually been completed. Some street names have changed like "Confederate" to "United"

Links to Infra Installation Dates (move to an excel sheet):
- https://www.letspropelatl.org/infra-tracker

In [ ]:
#TODO, resolve difference between the old inventory and the old as some streets had their facilities upgraded recently (Edgewood) and (Confederate/United)
coa = gpd.read_file('D:/RAW/City_of_Atlanta/coa_bike_facilities_new.geojson',mask=studyarea_geo)
coa.to_crs(config['projected_crs_epsg'],inplace=True)

# only keep existing and ones where the year is defined
coa = coa[(coa['Status']=='Existing') & coa['YearInstalled'].notna()]
# remove uncessary columns
coa.drop(columns=['GlobalID','Shape__Length','LengthMi','CrossSectionNotes','Status'],inplace=True)

#rename the id/year column
coa.rename(columns={'OBJECTID':'id','YearInstalled':'year'},inplace=True)

#mark facilities that need to be dated
coa['need_date'] = (coa['year'] >= 2012) & (coa['year'] <= 2016)

# rename columns for consistency
coa.columns = ['coa_'+col.lower() if col != 'geometry' else 'geometry' for col in coa.columns.tolist()]

# convert facility type to OSM (use highest protection if two different types)
osm_types = ['sharrow','bike lane','buffered bike lane','cycletrack','multi use path']
coa_conversion = {
       'Protected Bike Lane': osm_types[3], 
       'Protected Bike Lane / Bike Lane': osm_types[3],
       'Two-Way Cycle Track': osm_types[3], 
       'Uphill Bike Lane / Downhill Sharrows': osm_types[1],
       'Sharrows': osm_types[0], 
       'Bike Lane': osm_types[1], 
       'Bike Lane ': osm_types[1],
       'Uphill Buffered Bike Lane / Downhill Sharrows': osm_types[2],
       'Buffered Bike Lane': osm_types[2], 
       'Buffered Contraflow Bike Lane / Bike Lane': osm_types[1],
       'Shared-Use Path': osm_types[4], 
       'Neighborhood Greenway': osm_types[0], 
       'Bike Lane / Sharrows': osm_types[1],
       'Shared-Use Path / Bike Lane': osm_types[4], 
       'Buffered Bike Lane / Bike Lane': osm_types[2],
       'Buffered Bike Lane / Shared-Use Path': osm_types[4],
       'Shared-Use Path / Sharrows': osm_types[4],
       'Uphill Protected Bike Lane / Downhill Sharrows': osm_types[3], 
       'Shared Path': osm_types[4]
}
coa['coa_osm_type'] = coa['coa_facilitytype'].map(coa_conversion)

#export
coa.to_file(export_fp/'reference_layers.gpkg',layer='coa')

## Atlanta Regional Commission

In [ ]:
arc = gpd.read_file('D:\RAW\Atlanta_Regional_Comission\Bike_Ped_Trail_Inventory_January2024.geojson',mask=studyarea_geo)
arc.to_crs(config['projected_crs_epsg'],inplace=True)

# the only na values are OTP
#arc[arc['Year_2'].isna()].explore()

# remove post 2016
arc = arc[arc['Year_2'] < 2016]

#clean the columns
arc = arc[['OBJECTID_1','Name', 'spec','Width', 'Material', 'Year_2', 'geometry']]

#rename the id column
arc.rename(columns={'OBJECTID_1':'id','Year_2':'year'},inplace=True)

#mark facilities that need to be dated
arc['need_date'] = (arc['year'] >= 2012) & (arc['year'] <= 2016)

# rename columns for consistency
arc.columns = ['arc_'+col.lower() if col != 'geometry' else 'geometry' for col in arc.columns.tolist()]

# remove these facilities
remove = ['Paved shoulder','Park Trail','Campus Path']
arc = arc[arc['arc_spec'].isin(remove)==False]

# convert to osm name
arc_conversion = {
    'Hard surface multi-use path': 'multi use path',
    'Protected bike lane': 'cycletrack',
    'Uphill bike lanes / downhill sharrows': 'bike lane',
    'Conventional bike lane': 'bike lane',
    'Buffered bike lane': 'buffered bike lane',
    'Bike lane with parking': 'bike lane',
    'Buffered bike lane (BUS ONLY lane)': 'buffered bike lane',
    #'Paved shoulder': 'bike lane', # consider dumping these
    'Two way cycle track': 'cycletrack',
    'Bike lane': 'bike lane',
    'Buffered/bike lane': 'buffered bike lane',
    'Shared use path or greenway': 'multi use path',
    'Side path': 'multi use path',
    'Bike lane and Side path': 'bike lane',
    'Uphill bike lane/downhill sharrow': 'bike lane',
    'Side Path': 'multi use path',
    'Climbing lane': 'bike lane',
    'Sidepath': 'multi use path',
    #'Park Trail': 'multi use path',
    'Bike Lane with parking': 'bike lane',
    #'Campus Path': 'multi use path',
    'Bike Lane': 'bike lane'
    #'Park Path': 'multi use path'
}
arc['arc_osm_type'] = arc['arc_spec'].map(arc_conversion)

#export
arc.to_file(export_fp/'reference_layers.gpkg',layer='arc')

## Michael Garber (come back to later for the osm conversion)

In [ ]:
garber = gpd.read_file('D:/RAW/Michael Garber/data_for_reid.shp',mask=studyarea_geo)
garber.to_crs(config['projected_crs_epsg'],inplace=True)

# remove post 2016
garber = garber[(pd.to_datetime(garber['rbbn_dt']).apply(lambda row: row.year) <= 2016) | (garber['rbbn_dt'].isna())]

#clean the columns
garber = garber[['edge_id','infr_6_', 'in_6___', 'osm_nm_sm',
       'rbbn_dt', 'geometry']]

#rename
garber.columns = ['id','Infra1','Infra2','Name','Ribbon Date','geometry']

#mark facilities that need to be dated
garber['need_date'] = (pd.to_datetime(garber['Ribbon Date']).apply(lambda row: row.year) >= 2012) | (garber['Ribbon Date'].isna())

# rename columns for consistency
garber.columns = ['garber_'+col.lower() if col != 'geometry' else 'geometry' for col in garber.columns.tolist()]

#export
garber.to_file(export_fp/'reference_layers.gpkg',layer='garber')

In [ ]:
# garber['garber_infra1'].append(garber['garber_infra2']).unique().tolist()
# garber[garber['garber_infra1']=='off_street_trail_dirt'].explore()
# cycleways_osm['facility_fwd'].append(cycleways_osm['facility_rev']).unique().tolist()